In [1]:
#General packages
import numpy as np
from My_Jampy import JAM
import emcee

#MPI
from schwimmbad import MPIPool

#Constants and usefull packages
from astropy.cosmology import Planck15 as cosmo
from astropy.constants import G, M_sun, c
import astropy.units as u

#Autolens Model packages
import autolens as al
import autolens.plot as aplt

#Combined Model package
import CombinedModel

#Useful constants
metre2Mpc = (1*u.m).to(u.Mpc)/u.m           #Constant factor to convert metre to Mpc.
kg2Msun = (1*u.kg/M_sun)*u.solMass/u.kg     #Constant factor to convert kg to Msun

G_Mpc = G*(metre2Mpc)**3/kg2Msun            #Gravitational constant in Mpc³/(Msun s²)
c_Mpc = c*metre2Mpc                         #Speed of light in Mpc/s


#Dataset path
data_folder = "/home/carlos/Documents/GitHub/Master-Degree/Autolens_tests/autolens_workspace/Test_3/Simulation_data/"

In [2]:
#Reading MGE inputs
            #attention to units
surf_lum, sigma_lum, qobs_lum = np.loadtxt("JAM_Input.txt", unpack=True)          #MGE decomposition
surf_dm, sigma_dm , qobs_dm   = np.loadtxt("SDP81_pseudo-DM.txt", unpack=True)    #DM component
norm_psf, sigma_psf           = np.loadtxt("MUSE_Psf_model.txt", unpack=True)     #PSF
xbin, ybin, vrms              = np.loadtxt("vrms_data.txt", unpack=True)          #Vrms data

muse_pixsize = 0.2                            #Muse pixel size [arcsec/px]

z_lens   = 0.299                                    #Lens redshifth
z_source = 3.100                                    #Source redshift

#Angular diameter distances
D_l = cosmo.angular_diameter_distance(z_lens)                   #Lens              
D_s = cosmo.angular_diameter_distance(z_source)                 #Source
D_ls = cosmo.angular_diameter_distance_z1z2(z_lens, z_source)   #Lens to source

## Models inicialization

"""
    To inicialize the model, we set some random values for the parameters. But it's only necessary for initialize the model. During the non-linear search, this values will be updated constantly until the best fit.
"""  


   #This quantities are our unknown parameters
inc       = 75                              #Inclination [deg]
mbh       = 1e10                            #Mass of black hole [M_sun]
beta      = np.full_like(surf_lum, 0.3)     #Anisotropy
ml        = 10                              #Mass to light ratio [M_sun/L_sun]
mag_shear = 0.01                            #Shear magnitude
phi_shear = 100.0                             #Shear angle
rho_s     = 1e10                            #dark matter intensity
qdm       = np.full_like(qobs_dm, 0.5)      #dark matter axial ratio
gamma     = 1.0                             #Gamma

#--------------------------------------------------------------------------------------------------#
# JAMPY MODEL

Jam_model = JAM(ybin=ybin, xbin=xbin, inc=inc, distance=D_l.value, mbh=mbh, beta=beta, rms=vrms,
               normpsf=norm_psf, sigmapsf=sigma_psf*muse_pixsize, pixsize=muse_pixsize)

    #Add Luminosity component
Jam_model.luminosity_component(surf_lum=surf_lum, sigma_lum=sigma_lum,
                                qobs_lum=qobs_lum, ml=ml)
    #Add DM component
Jam_model.DM_component(surf_dm=rho_s * surf_dm, sigma_dm=sigma_dm, qobs_dm=qdm)


#--------------------------------------------------------------------------------------------------#
# PYAUTOLENS MODEL

imaging = al.Imaging.from_fits(
    image_path=f"{data_folder}/arcs_simulation.fits",
    noise_map_path=f"{data_folder}/noise_simulation.fits",
    psf_path=f"{data_folder}/psf_simulation.fits",
    pixel_scales=0.1,
)

mask = al.Mask.circular_annular(centre=(0.0, -0.2), inner_radius=1., outer_radius=2.3,
                          pixel_scales=imaging.pixel_scales, shape_2d=imaging.shape_2d) #Create a mask

masked_image = al.MaskedImaging(imaging=imaging, mask=mask, inversion_uses_border=True) #Masked image

mass_profile = al.mp.MGE()

    #Components
mass_profile.MGE_comps(z_l=z_lens, z_s=z_source, 
                   surf_lum=surf_lum, sigma_lum=sigma_lum, qobs_lum=qobs_lum, ml=ml,
                   mbh=mbh, surf_dm =rho_s * surf_dm, sigma_dm=sigma_dm, qobs_dm=qdm)

#--------------------------------------------------------------------------------------------------#
# COMBINED MODEL

    #Just remembering, by default the model does not include dark matter.
model = CombinedModel.Models(Jampy_model=Jam_model, mass_profile=mass_profile,
                             masked_imaging=masked_image, quiet=True)

model.mass_to_light(ml_kind='scalar')               #Setting gradient ML
model.beta(beta_kind='scalar')                      #Seting vector anisotropy
model.has_DM(a=True,filename="SDP81_pseudo-DM.txt") #Setting Dark matter component
#--------------------------------------------------------------------------------------------------#

In [3]:
def call_model(x0):
    print(x0)
    value =  model(x0)
    #print(value)
    if np.isfinite(value):
        return -value
    else:
        return 1e200

In [4]:
from scipy.optimize import shgo

In [6]:
#In order: ML, beta, inc, log_mbh, log_rho_s, qDM, mag_shear, phi_shear, gamma
bounds = [(1, 10), (-2, 2), (40, 90), (6, 12), (6, 12), (0.2, 1), (0, 0.1), (0, 179), (0.8, 1.2)]
import time

In [ ]:
start = time.time()

result = shgo(call_model, bounds, iters=100, sampling_method='sobol', options={'gtol': 1e-6, 'disp': True})

final = time.time()
print("Final time")
print(final - start)

Splitting first generation
Generating sampling points
[ 1.  -2.  40.   6.   6.   0.2  0.   0.   0.8]
-inf
[5.50e+00 0.00e+00 6.50e+01 9.00e+00 9.00e+00 6.00e-01 5.00e-02 8.95e+01
 1.00e+00]


/home/carlos/anaconda3/lib/python3.7/site-packages/jampy/legacy/jam_axi_rms.py:281: RuntimeWarning: divide by zero encountered in true_divide
  / signal.fftconvolve(mgeCar, kernel, mode='same')
/home/carlos/anaconda3/lib/python3.7/site-packages/jampy/legacy/jam_axi_rms.py:281: RuntimeWarning: invalid value encountered in true_divide
  / signal.fftconvolve(mgeCar, kernel, mode='same')


-129794.1432048371
[ 7.750e+00 -1.000e+00  7.750e+01  7.500e+00  1.050e+01  4.000e-01
  7.500e-02  4.475e+01  9.000e-01]
-31609.76596243182
[3.2500e+00 1.0000e+00 5.2500e+01 1.0500e+01 7.5000e+00 8.0000e-01
 2.5000e-02 1.3425e+02 1.1000e+00]
-inf
[ 4.37500e+00 -5.00000e-01  7.12500e+01  6.75000e+00  1.12500e+01
  9.00000e-01  1.25000e-02  1.11875e+02  8.50000e-01]
-inf
[ 8.875   1.5    46.25    9.75    8.25    0.5     0.0625 22.375   1.05  ]
-inf
[ 6.62500e+00 -1.50000e+00  5.87500e+01  8.25000e+00  6.75000e+00
  7.00000e-01  8.75000e-02  1.56625e+02  9.50000e-01]
-82782.3962519772
[2.1250e+00 5.0000e-01 8.3750e+01 1.1250e+01 9.7500e+00 3.0000e-01
 3.7500e-02 6.7125e+01 1.1500e+00]
-inf
[ 2.687500e+00 -7.500000e-01  5.562500e+01  1.012500e+01  9.375000e+00
  3.500000e-01  6.250000e-03  1.678125e+02  8.750000e-01]
-inf
[7.18750e+00 1.25000e+00 8.06250e+01 7.12500e+00 6.37500e+00 7.50000e-01
 5.62500e-02 7.83125e+01 1.07500e+00]
-106890.14350398301
[ 9.437500e+00 -1.750000e+00  6.812500e

-98646.56439411921
[1.14062500e+00 1.18750000e+00 8.76562500e+01 1.00312500e+01
 1.07812500e+01 9.37500000e-01 7.34375000e-02 1.59421875e+02
 1.01875000e+00]
-73726.49929327251
[ 1.21093750e+00 -4.06250000e-01  6.22656250e+01  1.08281250e+01
  7.26562500e+00  8.93750000e-01  3.98437500e-02  1.24460938e+02
  8.96875000e-01]
-30366.45667503406
[ 5.7109375   1.59375    87.265625    7.828125   10.265625    0.49375
  0.08984375 34.9609375   1.096875  ]
-196490.53084027744
[ 7.96093750e+00 -1.40625000e+00  7.47656250e+01  9.32812500e+00
  1.17656250e+01  6.93750000e-01  6.48437500e-02  1.69210938e+02
  9.96875000e-01]
-86505.21105522041
[3.46093750e+00 5.93750000e-01 4.97656250e+01 6.32812500e+00
 8.76562500e+00 2.93750000e-01 1.48437500e-02 7.97109375e+01
 1.19687500e+00]
-inf
[ 4.58593750e+00 -1.90625000e+00  8.10156250e+01  1.15781250e+01
  1.10156250e+01  3.93750000e-01  2.73437500e-02  1.25859375e+01
  8.46875000e-01]
-inf
[9.08593750e+00 9.37500000e-02 5.60156250e+01 8.57812500e+00
 8.

-33320.346398055706
[2.75781250e+00 1.90625000e+00 8.17968750e+01 1.09218750e+01
 8.48437500e+00 2.06250000e-01 7.89062500e-02 1.16070312e+02
 1.00312500e+00]
-79291.11461564037
[ 2.1953125  -1.84375    59.921875    9.046875    8.109375    0.25625
  0.07265625 37.7578125   0.928125  ]
-inf
[6.69531250e+00 1.56250000e-01 8.49218750e+01 6.04687500e+00
 1.11093750e+01 6.56250000e-01 2.26562500e-02 1.27257812e+02
 1.12812500e+00]
-inf
[ 8.94531250e+00 -8.43750000e-01  7.24218750e+01  1.05468750e+01
  9.60937500e+00  4.56250000e-01  4.76562500e-02  8.25078125e+01
  8.28125000e-01]
-56498.37805732468
[4.44531250e+00 1.15625000e+00 4.74218750e+01 7.54687500e+00
 6.60937500e+00 8.56250000e-01 9.76562500e-02 1.72007812e+02
 1.02812500e+00]
-inf
[ 3.32031250e+00 -3.43750000e-01  7.86718750e+01  9.79687500e+00
  1.03593750e+01  9.56250000e-01  6.01562500e-02  1.04882812e+02
  9.78125000e-01]
-19412.96956630326
[7.82031250e+00 1.65625000e+00 5.36718750e+01 6.79687500e+00
 7.35937500e+00 5.56250000

...

-154329.6697350403
[ 4.25799561e+00 -1.74389648e+00  4.66864014e+01  7.90759277e+00
  9.57824707e+00  5.59423828e-01  9.68566895e-02  1.40969055e+02
  8.89428711e-01]
-inf
[8.75799561e+00 2.56103516e-01 7.16864014e+01 1.09075928e+01
 6.57824707e+00 9.59423828e-01 4.68566895e-02 5.14690552e+01
 1.08942871e+00]
-44426.36252004522
[ 6.50799561e+00 -7.43896484e-01  8.41864014e+01  6.40759277e+00
  8.07824707e+00  3.59423828e-01  2.18566895e-02  9.62190552e+01
  9.89428711e-01]
-inf
[ 2.00799561  1.25610352 59.18640137  9.40759277 11.07824707  0.75942383
  0.07185669  6.71905518  1.18942871]
-74562.1871498306
[ 1.72674561e+00 -6.18896484e-01  4.19989014e+01  9.97009277e+00
  9.76574707e+00  9.84423828e-01  3.12316895e-02  4.58753052e+01
  1.15192871e+00]
-inf
[6.22674561e+00 1.38110352e+00 6.69989014e+01 6.97009277e+00
 6.76574707e+00 5.84423828e-01 8.12316895e-02 1.35375305e+02
 9.51928711e-01]
-inf
[ 8.47674561e+00 -1.61889648e+00  7.94989014e+01  1.14700928e+01
  8.26574707e+00  7.844238

-158548.20452771816
[ 4.75018311e+00 -1.40014648e+00  4.00457764e+01  7.11071777e+00
  7.46887207e+00  6.90673828e-01  5.07629395e-02  9.76174927e+01
  8.86303711e-01]
-inf
[9.25018311e+00 5.99853516e-01 6.50457764e+01 1.01107178e+01
 1.04688721e+01 2.90673828e-01 7.62939453e-04 8.11749268e+00
 1.08630371e+00]
-inf
[ 7.00018311e+00 -4.00146484e-01  7.75457764e+01  8.61071777e+00
  1.19688721e+01  8.90673828e-01  2.57629395e-02  1.42367493e+02
  9.86303711e-01]
-147200.75737363138
[ 2.50018311  1.59985352 52.54577637 11.61071777  8.96887207  0.49067383
  0.07576294 52.86749268  1.18630371]
-inf
[ 3.06268311e+00 -1.65014648e+00  8.69207764e+01  1.04857178e+01
  7.84387207e+00  4.40673828e-01  6.95129395e-02  1.53554993e+02
  8.61303711e-01]
-18781.608725757374
[7.56268311e+00 3.49853516e-01 6.19207764e+01 7.48571777e+00
 1.08438721e+01 8.40673828e-01 1.95129395e-02 6.40549927e+01
 1.06130371e+00]
-35276.312141751914
[ 9.81268311e+00 -6.50146484e-01  4.94207764e+01  1.19857178e+01
  9.343

-60118.20923118938
[4.78533936e+00 1.39672852e+00 5.54754639e+01 1.04154053e+01
 1.02110596e+01 8.12548828e-01 1.75598145e-02 1.12301086e+02
 8.97241211e-01]
-inf
[ 3.66033936e+00 -1.03271484e-01  7.42254639e+01  8.16540527e+00
  6.46105957e+00  9.12548828e-01  3.00598145e-02  1.34676086e+02
  1.14724121e+00]
-25461.655485096977
[ 8.16033936  1.89672852 49.22546387 11.16540527  9.46105957  0.51254883
  0.08005981 45.17608643  0.94724121]
-inf
[ 5.91033936e+00 -1.10327148e+00  6.17254639e+01  6.66540527e+00
  1.09610596e+01  7.12548828e-01  5.50598145e-02  8.99260864e+01
  1.04724121e+00]
-inf
[1.41033936e+00 8.96728516e-01 8.67254639e+01 9.66540527e+00
 7.96105957e+00 3.12548828e-01 5.05981445e-03 4.26086426e-01
 8.47241211e-01]
-74655.53661719972
[ 1.55096436e+00 -1.65771484e-01  4.53192139e+01  7.13415527e+00
  9.74230957e+00  8.25048828e-01  6.91223145e-02  1.59847961e+02
  1.02849121e+00]
-inf
[6.05096436e+00 1.83422852e+00 7.03192139e+01 1.01341553e+01
 6.74230957e+00 4.25048828e-

-86891.96282044065
[ 8.79315186e+00 -2.59521484e-01  8.63348389e+01  7.55603027e+00
  9.50793457e+00  7.31298828e-01  8.55285645e-02  6.89495239e+01
  1.07536621e+00]
-45662.355290635205
[4.29315186e+00 1.74047852e+00 6.13348389e+01 1.05560303e+01
 6.50793457e+00 3.31298828e-01 3.55285645e-02 1.58449524e+02
 8.75366211e-01]
-inf
[ 5.41815186e+00 -7.59521484e-01  6.75848389e+01  6.80603027e+00
  1.02579346e+01  2.31298828e-01  2.30285645e-02  9.13245239e+01
  1.12536621e+00]
-inf
[ 9.91815186  1.24047852 42.58483887  9.80603027  7.25793457  0.63129883
  0.07302856  1.82452393  0.92536621]
-inf
[ 7.66815186e+00 -1.75952148e+00  5.50848389e+01  8.30603027e+00
  8.75793457e+00  4.31298828e-01  9.80285645e-02  1.36074524e+02
  1.02536621e+00]
-inf
[3.16815186e+00 2.40478516e-01 8.00848389e+01 1.13060303e+01
 1.17579346e+01 8.31298828e-01 4.80285645e-02 4.65745239e+01
 8.25366211e-01]
-inf
[ 2.60565186e+00 -9.52148438e-03  6.44598389e+01  1.01810303e+01
  1.06329346e+01  8.81298828e-01  4.27

-61774.71933783489
[ 3.85369873e+00 -1.31420898e+00  4.50262451e+01  9.32556152e+00
  1.18399658e+01  3.45361328e-01  2.67395020e-02  4.34280396e+01
  1.12770996e+00]
-inf
[8.35369873e+00 6.85791016e-01 7.00262451e+01 6.32556152e+00
 8.83996582e+00 7.45361328e-01 7.67395020e-02 1.32928040e+02
 9.27709961e-01]
-inf
[ 6.10369873e+00 -3.14208984e-01  8.25262451e+01  1.08255615e+01
  7.33996582e+00  5.45361328e-01  5.17395020e-02  8.81780396e+01
  1.02770996e+00]
-11157.373823286438
[1.60369873e+00 1.68579102e+00 5.75262451e+01 7.82556152e+00
 1.03399658e+01 9.45361328e-01 1.73950195e-03 1.77678040e+02
 8.27709961e-01]
-16307.7075750802
Constructing Gabrial graph and minimizer pool


In [11]:
print(result)

Help on function shgo in module scipy.optimize._shgo:

shgo(func, bounds, args=(), constraints=None, n=100, iters=1, callback=None, minimizer_kwargs=None, options=None, sampling_method='simplicial')
    Finds the global minimum of a function using SHG optimization.
    
    SHGO stands for "simplicial homology global optimization".
    
    Parameters
    ----------
    func : callable
        The objective function to be minimized.  Must be in the form
        ``f(x, *args)``, where ``x`` is the argument in the form of a 1-D array
        and ``args`` is a tuple of any additional fixed parameters needed to
        completely specify the function.
    bounds : sequence
        Bounds for variables.  ``(min, max)`` pairs for each element in ``x``,
        defining the lower and upper bounds for the optimizing argument of
        `func`. It is required to have ``len(bounds) == len(x)``.
        ``len(bounds)`` is used to determine the number of parameters in ``x``.
        Use ``None`` f